<a href="https://colab.research.google.com/github/Antoine723/ProjetNLP/blob/main/ProjetNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [Import bibliothèques et fichiers utiles](#0)
- [Récupération commentaires API Yelp](#1)
- [Traitement](#2)
- [Création vocabulaire grâce aux données](#3)
- [Transformation des mots en indices](#4)
- [Création modèles](#5)
- [Pipeline](#6)
- [Test des modèles sur notre jeu de données](#7)
- [Détermination du seuil selon les données observées](#8)

<a name="0"></a>
# Import bibliothèques et fichiers utiles

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf

from gdrive.MyDrive.ProjetNLP.rnn_utils import *

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
pip install --pre gql[all]

     |████████████████████████████████| 81kB 2.3MB/s 
     |████████████████████████████████| 194kB 7.5MB/s 
     |████████████████████████████████| 296kB 43.9MB/s 
     |████████████████████████████████| 1.3MB 37.6MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 143kB 43.5MB/s 
  Created wheel for gql: filename=gql-3.0.0a5-py2.py3-none-any.whl size=32709 sha256=d04483bbb684e7c625da9dedf50490a7c8b7f5f7a9a399203b1f5104170e32be
  Stored in directory: /root/.cache/pip/wheels/59/f4/5b/d4678b0d7c58b5b71cb44b849833ab4e6a0a9e4b6aad41b016
Successfully built gql


In [4]:
import requests
import json

<a name="1"></a>
# Récupération commentaires API Yelp

In [ ]:
api_key='MI6JGndpuW3Kyc9cIjj5of9AhZUKET4JlGdRsiBnGZjL2ofbChjG0ksWdjXrHvdRN3cHosvu6bkDcvdOUCc5A_m-IAciq5kpjSqAxt5euSYLzB0q0vvWqmuyrI86YHYx'
headers = {'Authorization': 'Bearer %s' % api_key}

url = "https://api.yelp.com/v3/businesses/FEVQpbOPOwAPNIgO7D3xxw/reviews"
req = requests.get(url, headers=headers)

dataset=json.loads(req.text)["reviews"]
x_test=[]
for i in range(len(dataset)):
  x_test.append(dataset[i]["text"])
x_test2=x_test.copy()
print(x_test)
print(x_test2)

["Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...", 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those crinkle cuts!) and a shake. Yum. Tbh,...', 'Before I moved to NY, my first ever trip to the city was in 2005 - this was when Shake Shack had this one lone original location, and it was an event to go...']
["Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...", 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those crinkle cuts!) and a shake. Yum. Tbh,...', 'Before I moved to NY, my first ever trip to the city was in 2005 - this was when Shake Shack had this one lone original location, and it was an event to go...']


<a name="2"></a>
# Traitement

In [ ]:
def processing(data):
  if (isinstance(data,str)):
    #MINUSCULES
    data=data.lower()
    #TOKENIZATION
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    data=tokenizer.tokenize(data)

    #STOPWORDS
    data=[w for w in data if not w in list(nltk.corpus.stopwords.words('english'))]
    #STEMMING
    stemmer=LancasterStemmer()
    data=[stemmer.stem(w) for w in data]

    #LEMMATIZATION
    Word_Lemmatizer = WordNetLemmatizer()
    data=[Word_Lemmatizer.lemmatize(w) for w in data]

  elif (isinstance(data,list)):
    for i in range(len(data)):
      data[i]=data[i].lower()

    #TOKENIZATION
    tokenizer = nltk.RegexpTokenizer(r'\w+')

    for i in range(len(data)):
      data[i]=tokenizer.tokenize(data[i])

    #STOPWORDS
    for i in range(len(data)):
      data[i]=[w for w in data[i] if not w in list(nltk.corpus.stopwords.words('english'))]
        

    #STEMMING
    stemmer=LancasterStemmer()
    for i in range(len(data)):
      data[i]=[stemmer.stem(w) for w in data[i]]

    #LEMMATIZATION
    Word_Lemmatizer = WordNetLemmatizer()
    for i in range(len(data)):
      data[i]=[Word_Lemmatizer.lemmatize(w) for w in data[i]]
    # print(data)

  else :
    #MINUSCULES
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=data[i][j].lower()

    #TOKENIZATION
    tokenizer = nltk.RegexpTokenizer(r'\w+')

    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=tokenizer.tokenize(data[i][j])

    #STOPWORDS
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=[w for w in data[i][j] if not w in list(nltk.corpus.stopwords.words('english'))]
        

    #STEMMING
    stemmer=LancasterStemmer()
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=[stemmer.stem(w) for w in data[i][j]]

    #LEMMATIZATION
    Word_Lemmatizer = WordNetLemmatizer()
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=[Word_Lemmatizer.lemmatize(w) for w in data[i][j]]
    # print(data)

  return data

<a name="3"></a>
# Création du vocabulaire grâce aux données

In [ ]:
from collections import defaultdict
def defineVocab(data):
  v=set()
  for i in range(len(data)):
    for j in range(len(data[i])):
      for h in range(len(data[i][j])):
        v.add(data[i][j][h])

  vocab = defaultdict(lambda: 0)
  vocab['<PAD>'] = 1

  j=2
  for i in v:
    vocab[i]=j
    j+=1
  return vocab

<a name="4"></a>
# Transformation des mots en indices

In [ ]:
def transformWords(data,vocab):
  matrixData=[]
  maxLen=0
  if (isinstance(data,list)):
    if (len(data)>1):
      for i in range(len(data)):
        tmp=[]
        for j in data[i]:
          if (j not in vocab.keys()):
            tmp.append(0)
          else:
            tmp.append(vocab[j])
        matrixData.append(tmp)
    else:
      for i in data:
        if (i not in vocab.keys()):
          matrixData.append(0)
        else:
          matrixData.append(vocab[i])
  else:
    for i in range(len(data)):
      for j in range(len(data[i])):
        if (maxLen < len(data[i][j])):
          maxLen= len(data[i][j])

    for i in range(len(data)):
      for j in range(len(data[i])):
        tmp=[]
        for k in range(maxLen):
          if (k >= len(data[i][j])):
            tmp.append(vocab['<PAD>'])
          else:
            tmp.append(vocab[data[i][j][k]])
        matrixData.append(tmp)
  return matrixData

<a name="5"></a>
# Création modèles

In [ ]:
def buildModelLSTM(vocab):
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim=len(vocab)+1,output_dim=1))
  model.add(tf.keras.layers.LSTM(128,activation="relu"))
  model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
  return model

In [ ]:
def buildModelSimpleRNN(vocab):
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim=len(vocab)+1,output_dim=1))
  model.add(tf.keras.layers.SimpleRNN(128,activation="relu"))
  model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
  return model

In [ ]:
def fitModel(data,model,negSize,posSize): #negSize et posSize = taille des données prises en compte pour le fit, respectivement des commentaires négatifs et positifs
# Ex : si on crée le modèle en lui donnant 20 commentaires négatifs et 20 positifs, negSize=20 et posSize=20
  y_train=np.concatenate([np.zeros((negSize,1)),np.ones((posSize,1))])
  model.compile(loss='binary_crossentropy',optimizer="adam")
  model.fit(data,y_train,epochs=50)

<a name="6"></a>
# Pipeline

In [ ]:
import os
def final_LSTM(sentence):
  threshold=0.2
  model_file="LSTM_Model"
  if (not os.path.isdir("gdrive/MyDrive/ProjetNLP/"+model_file)):
    df=pd.read_csv('gdrive/MyDrive/ProjetNLP/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",header=None)
    x_train=np.array([df.iloc[0:1000][5],df.iloc[800000:801000][5]])
    x_train=processing(x_train)
    vocab=defineVocab(x_train)
    df=pd.DataFrame(vocab.items(),columns=["Word","Id"])
    df.to_csv("gdrive/MyDrive/ProjetNLP/vocab")
    x_train=transformWords(x_train,vocab)
    x_train=np.array(x_train)
    model=buildModelLSTM(vocab)
    fitModel(x_train,model,1000,1000)
    model.save("gdrive/MyDrive/ProjetNLP/"+model_file)
  else:
    model=tf.keras.models.load_model("gdrive/MyDrive/ProjetNLP/"+model_file)
    vocab=pd.read_csv("gdrive/MyDrive/ProjetNLP/vocab").set_index("Word").to_dict()["Id"]
  # return model
  sentence=processing(sentence)
  sentence=transformWords(sentence,vocab)
  sentence=np.array(sentence)
  if (len(sentence.shape)==1 and len(sentence)==1):
    if (model.predict(np.expand_dims(sentence,axis=0))<threshold):
      commentTypes="Negative"
    else:
      commentTypes="Positive"
  else:
    commentTypes=[]
    for i in range(len(sentence)):
      if (model.predict(np.expand_dims(sentence[i],axis=0))<threshold):
        commentTypes.append("Negative")
      else:
        commentTypes.append("Positive")
  return commentTypes




In [ ]:
import os
def final_SimpleRNN(sentence):
  threshold=0.4
  model_file="SimpleRNN_Model"
  if (not os.path.isdir("gdrive/MyDrive/ProjetNLP/"+model_file)):
    df=pd.read_csv('gdrive/MyDrive/ProjetNLP/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",header=None)
    x_train=np.array([df.iloc[0:1000][5],df.iloc[800000:801000][5]])
    x_train=processing(x_train)
    vocab=defineVocab(x_train)
    df=pd.DataFrame(vocab.items(),columns=["Word","Id"])
    df.to_csv("gdrive/MyDrive/ProjetNLP/vocab")
    x_train=transformWords(x_train,vocab)
    x_train=np.array(x_train)
    model=buildModelSimpleRNN(vocab)
    fitModel(x_train,model,1000,1000)
    model.save("gdrive/MyDrive/ProjetNLP/"+model_file)
  else:
    model=tf.keras.models.load_model("gdrive/MyDrive/ProjetNLP/"+model_file)
    vocab=pd.read_csv("gdrive/MyDrive/ProjetNLP/vocab").set_index("Word").to_dict()["Id"]
  # return model
  sentence=processing(sentence)
  sentence=transformWords(sentence,vocab)
  sentence=np.array(sentence)
  if (len(sentence.shape)==1 and len(sentence)==1):
    if (model.predict(np.expand_dims(sentence,axis=0))<threshold):
      commentTypes="Negative"
    else:
      commentTypes="Positive"
  else:
    commentTypes=[]
    for i in range(len(sentence)):
      if (model.predict(np.expand_dims(sentence[i],axis=0))<threshold):
        commentTypes.append("Negative")
      else:
        commentTypes.append("Positive")
  return commentTypes




<a name="7"></a>
# Test des modèles sur notre jeu de données

In [ ]:
final_LSTM(x_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


['Positive', 'Positive', 'Positive']

In [ ]:
final_SimpleRNN(x_test2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


['Positive', 'Positive', 'Positive']

<a name="8"></a>
# Détermination du seuil selon les données observées

In [ ]:
from sklearn.metrics import f1_score
def defineThreshold(datas,model):
  y_train=np.concatenate([np.zeros((1000,1)),np.ones((1000,1))])
  seuils=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
  F1_score=[]
  predict=model.predict(datas)
  for seuil in seuils:
    predictions=[]
    for i in range(len(datas)):
      if (predict[i]<seuil):
        predictions.append(0)
      else:
        predictions.append(1)
    F1_score.append(f1_score(y_train,predictions))
  print(F1_score)
  seuil=seuils[F1_score.index(max(F1_score))]
  print(seuil)
  return seuil

In [ ]:
# defineThreshold(np.array(dat),Simple_RNN)